# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("home_decoration_data_glm4.txt", encoding="utf-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题]：你们的家装设计风格有哪些？\n[销售回答]：我们提供多种设计风格，包括现代简约、中式古典、欧式奢华等，可以根据您的喜好和需求为您量身定制。')

In [6]:
len(docs)

100

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "材料环保吗？"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]：你们的施工过程中会产生异味吗？
[销售回答]：我们会使用环保材料，尽量减少施工过程中的异味。

[客户问题]：你们的装修材料质量如何？
[销售回答]：我们使用的装修材料都是经过严格筛选的优质产品，保证环保、健康、耐用，让您和家人放心入住。

[客户问题]：你们提供装修后的环保检测服务吗？ 
[销售回答]：是的，我们非常重视您和您家人的健康，装修完成后会提供室内空气质量检测服务，确保装修材料和环境安全达标，让您住得安心。

[客户问题]：你们的装修服务包括环保检测吗？
[销售回答]：我们非常重视环保，装修完成后会提供室内空气质量检测服务，确保您的居住环境安全健康。



In [11]:
db.save_local("home_decoration_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f6a7ffedc50>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：你们的施工过程中会产生异味吗？
[销售回答]：我们会使用环保材料，尽量减少施工过程中的异味。

[客户问题]：你们的装修材料质量如何？
[销售回答]：我们使用的装修材料都是经过严格筛选的优质产品，保证环保、健康、耐用，让您和家人放心入住。

[客户问题]：你们提供装修后的环保检测服务吗？ 
[销售回答]：是的，我们非常重视您和您家人的健康，装修完成后会提供室内空气质量检测服务，确保装修材料和环境安全达标，让您住得安心。



In [15]:
docs = topK_retriever.get_relevant_documents("你们装修要多久？")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：你们的工期需要多久？
[销售回答]：我们的工期根据工程量和施工难度而定，但我们会合理安排，确保按时完工。

[客户问题]：你们的装修效果能保证吗？
[销售回答]：我们的设计团队会根据您的需求进行精心设计，施工团队严格按照设计方案进行施工，保证装修效果符合您的预期。

[客户问题]：你们的施工过程中会有改动吗？
[销售回答]：我们会根据实际情况进行合理的改动，确保装修效果。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：你们的施工过程中会产生异味吗？
[销售回答]：我们会使用环保材料，尽量减少施工过程中的异味。

[客户问题]：你们的装修材料质量如何？
[销售回答]：我们使用的装修材料都是经过严格筛选的优质产品，保证环保、健康、耐用，让您和家人放心入住。



### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'[客户问题]：你们的施工过程中会产生异味吗？\n[销售回答]：我们会使用环保材料，尽量减少施工过程中的异味。'

In [21]:
docs[0].page_content.split("[销售回答]：")

['[客户问题]：你们的施工过程中会产生异味吗？\n', '我们会使用环保材料，尽量减少施工过程中的异味。']

In [22]:
ans = docs[0].page_content.split("[销售回答]：")[-1]

In [23]:
ans

'我们会使用环保材料，尽量减少施工过程中的异味。'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答]：")[-1] for doc in docs]

    return ans_list

In [25]:
query = "材料环保吗？"

print(sales(query))

['我们会使用环保材料，尽量减少施工过程中的异味。', '我们使用的装修材料都是经过严格筛选的优质产品，保证环保、健康、耐用，让您和家人放心入住。']


In [26]:
print(sales(query, 0.75))

['我们会使用环保材料，尽量减少施工过程中的异味。', '我们使用的装修材料都是经过严格筛选的优质产品，保证环保、健康、耐用，让您和家人放心入住。', '是的，我们非常重视您和您家人的健康，装修完成后会提供室内空气质量检测服务，确保装修材料和环境安全达标，让您住得安心。', '我们非常重视环保，装修完成后会提供室内空气质量检测服务，确保您的居住环境安全健康。']


In [27]:
query = "材料环保吗？"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: ['我们会使用环保材料，尽量减少施工过程中的异味。', '我们使用的装修材料都是经过严格筛选的优质产品，保证环保、健康、耐用，让您和家人放心入住。']

score:0.75 ans: ['我们会使用环保材料，尽量减少施工过程中的异味。', '我们使用的装修材料都是经过严格筛选的优质产品，保证环保、健康、耐用，让您和家人放心入住。', '是的，我们非常重视您和您家人的健康，装修完成后会提供室内空气质量检测服务，确保装修材料和环境安全达标，让您住得安心。', '我们非常重视环保，装修完成后会提供室内空气质量检测服务，确保您的居住环境安全健康。']

score:0.5 ans: ['我们会使用环保材料，尽量减少施工过程中的异味。', '我们使用的装修材料都是经过严格筛选的优质产品，保证环保、健康、耐用，让您和家人放心入住。', '是的，我们非常重视您和您家人的健康，装修完成后会提供室内空气质量检测服务，确保装修材料和环境安全达标，让您住得安心。', '我们非常重视环保，装修完成后会提供室内空气质量检测服务，确保您的居住环境安全健康。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [29]:
qa_chain({"query": "材料一定要环保？"})

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': '材料一定要环保？',
 'result': '是的，对于许多项目来说，使用环保材料是一种常见的做法。环保材料可以减少对环境和人体健康的负面影响，并且在施工过程中能够减少异味的产生。'}

In [30]:
qa_chain({"query": "装修有哪些风格"})

{'query': '装修有哪些风格',
 'result': '我们提供多种设计风格，包括现代简约、中式古典、欧式奢华等，可以根据您的喜好和需求为您量身定制。'}

In [31]:
print(sales("装修有哪些风格"))

['我们提供多种设计风格，包括现代简约、中式古典、欧式奢华等，可以根据您的喜好和需求为您量身定制。']


## 加载 FAISS 向量数据库已有结果

In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("home_decoration_sale", OpenAIEmbeddings(),allow_dangerous_deserialization=True)

In [33]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

/root/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [34]:
qa_chain({"query": "我想要中式简约，符合年轻人得装修风格"})

{'query': '我想要中式简约，符合年轻人得装修风格',
 'result': '中式简约是一种非常适合年轻人的装修风格，它融合了传统中式元素和现代简约风格，给人一种清新、自然的感觉。这种风格通常注重线条简洁、色彩淡雅，同时保留了一些传统中式家居元素，如木质家具、自然材料和文化装饰等。你可以选择一些简约但具有中式特色的家具，如实木家具或者带有中式图案的软装饰品，来营造出与众不同的空间氛围。'}

In [35]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [36]:
qa_chain({"query": "我想要中式简约，符合年轻人得装修风格"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想要中式简约，符合年轻人得装修风格',
 'result': '根据你的需求，中式简约可能是个不错的选择。这种风格融合了传统中式元素，但又不失现代简约的特点，适合年轻人的审美。可以选择一些简洁而又具有中式特色的家具和装饰，如红木家具、汉白玉等，同时注重空间的整体布局和色彩的搭配，营造出舒适、清新的居家氛围。'}

In [37]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [38]:
result = qa_chain({"query": "我想要中式简约，符合年轻人得装修风格"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [39]:
result

{'query': '我想要中式简约，符合年轻人得装修风格',
 'result': '我们公司有适合年轻人的中式简约风格，它将传统的中式元素与简约现代的设计理念相结合，给人一种清新、自然的感觉。这种风格通常会注重线条的简洁与空间的通透感，同时保留了一些传统的中式装饰元素，如木质家具、青花瓷等，但以简约的方式呈现，适合年轻人追求简约时尚的品味。',
 'source_documents': [Document(page_content='[客户问题]：你们的家装设计风格有哪些？\n[销售回答]：我们提供多种设计风格，包括现代简约、中式古典、欧式奢华等，可以根据您的喜好和需求为您量身定制。')]}